# Mise en pratique [regression linéaire]

## 1. Exploration des données

## 1.1 calculer la vitesse moyenne de chaque trajet (en km/h) 

## 1.2 calculer le nombre de trajets effectués en fonction du jour de la semaine 

## 1.3 calculer le nombre de trajets effectués en fonction de l’horaire de la journée par tranche de 4h.

## 1.4 calculer le nombre de km parcourus par jour de la semaine

## 2. Choix du modèle de machine learning

## 3. Préparation des données

### 3.a échantillonage

### 3.b Sélection de la variable cible et des variables à explicatives

## 4. Division du jeu de données en training/validation/testing sets

## 5. Apprentissage sur le training set

#### On implémente la regression linéaire

#### On obtient ainsi les parmètres β1 et β0 de notre modèle

#### On peut à présent réaliser des prédictions

#### enfin on peut représenter notre modèle linéaire

## 6. Evaluation du modèle sur le testing set

#### évaluation à partir du R2

#### évaluation à partir du RMSE

#### évaluation graphique